In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.regression_models as RegressionModels
import core.assignment_models as AssignmentModels
import core
import keras
import core.regression_models.RegressionTransformer as RegressionTransformer

plt.rcParams["font.size"] = 14

PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=4_000_000
)

X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)

2025-12-17 23:31:27.906871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766010687.930839  149474 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766010687.938810  149474 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766010687.956985  149474 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766010687.957003  149474 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766010687.957006  149474 computation_placer.cc:177] computation placer alr

In [ ]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformer(data_config)

In [ ]:
Transformer.build_model(
    hidden_dim=128,
    num_heads=8,
    central_transformer_stack_size=4,
    regression_transformer_stack_size=4,
    dropout_rate=0.1,
    compute_HLF=True,
    log_variables=True,
)

Transformer.adapt_normalization_layers(X_train)

Transformer.compile_model(
    optimizer=keras.optimizers.Lion(learning_rate=1e-4),
    loss={
        "normalized_regression": keras.losses.MeanSquaredLogarithmicError(),
        "assignment": core.utils.AssignmentLoss(),
    },
    metrics={
        "normalized_regression": core.utils.RegressionDeviation(),
        "assignment": core.utils.AssignmentAccuracy(),
    },
    loss_weights={
        "normalized_regression": 1.0,
        "assignment": 5.0,
    },
)


In [ ]:
Transformer.train_model(
    X_train,
    y_train,
    batch_size=1024,
    epochs=50,
    copy_data=True,
)

In [ ]:
Transformer.save_model(MODEL_DIR + "model.keras")